In [ ]:
import numpy as np
import pandas as pd
import sqlite3

## Задача 1. Конвертация
### Исходные данные
Таблица `operations` содержит операции клиентов

In [ ]:
operations = pd.DataFrame([
    ['100100', '2022-04-29 00:00:10.000', 50000, 'RUR', '000000'],
    ['100200', '2022-05-01 10:30:30.000', -20000, 'RUR', '230000']
], columns=['customer_id', 'operation_dttm',
            'operation_amt', 'currency_id', 'terminal_id'])

In [ ]:
operations['operation_dttm'] = pd.to_datetime(operations['operation_dttm'])

In [ ]:
operations

,customer_id,operation_dttm,operation_amt,currency_id,terminal_id
0,100100,2022-04-29 00:00:10,50000,RUR,000000
1,100200,2022-05-01 10:30:30,-20000,RUR,230000


Таблица `fx_rates` содержит курсы валют, установленные в рабочие дни

In [ ]:
fx_rates = pd.DataFrame([
    # ['2022-04-29', 73.5589, 'EUR'],
    ['2022-04-30', 71.0237, 'USD'],
    ['2022-04-30', 74.5589, 'EUR'],
    ['2022-05-05', 69.4160, 'USD'],
    ['2022-05-05', 72.7815, 'EUR']
], columns=['rate_dt', 'fx_rate', 'currency_id'])

In [ ]:
fx_rates['rate_dt'] = pd.to_datetime(fx_rates['rate_dt'])

## Задача
Напишите запрос, выводящий эквивалент в EUR для каждой операции клиента. Для операций в нерабочие дни используется курс, установленный в ближайший предшествующий рабочий день.

In [ ]:
pd.merge_asof(operations, fx_rates.query('currency_id == "EUR"'),
              left_on='operation_dttm', right_on='rate_dt',
              direction='backward') \
    .assign(in_euro=lambda df_:  df_['operation_amt'] / df_['fx_rate'])

,customer_id,operation_dttm,operation_amt,currency_id_x,terminal_id,rate_dt,fx_rate,currency_id_y,in_euro
0,100100,2022-04-29 00:00:10,50000,RUR,000000,2022-04-29,73.5589,EUR,679.727402
1,100200,2022-05-01 10:30:30,-20000,RUR,230000,2022-04-30,74.5589,EUR,-268.244301


In [ ]:
con = sqlite3.connect('db')
cur = con.cursor()

operations.to_sql('operations', con, index=False, if_exists='replace')
fx_rates.to_sql('fx_rates', con, index=False, if_exists='replace')

In [ ]:
def select(query):
  return pd.read_sql(query, con)

In [ ]:
query = '''
SELECT
  operation_dttm
  , operation_amt
  , currency_id  
  , operation_amt / (
                     SELECT fx_rate  
                     FROM fx_rates  
                     WHERE rate_dt = (
                                       SELECT MAX(rate_dt)  
                                       FROM fx_rates  
                                       WHERE rate_dt <= DATE(operation_dttm)
                                         AND currency_id = 'EUR' 
                                     )  
                           AND currency_id = 'EUR' 
                  ) AS amt_EUR  
FROM operations 
ORDER BY operation_dttm;'''

select(query)

,operation_dttm,operation_amt,currency_id,amt_EUR
0,2022-04-29 00:00:10,50000,RUR,NaN
1,2022-05-01 10:30:30,-20000,RUR,-268.244301


In [ ]:
query = '''
SELECT
  operation_dttm
  , operation_amt
  , currency_id
  , operation_amt / (
                      SELECT FIRST_VALUE(fx_rate) OVER(ORDER BY rate_dt DESC)
                      FROM fx_rates
                      WHERE rate_dt <= DATE(operation_dttm)
                        AND currency_id = 'EUR'
                    )  AS in_euro
FROM operations
'''
select(query)

,operation_dttm,operation_amt,currency_id,in_euro
0,2022-04-29 00:00:10,50000,RUR,NaN
1,2022-05-01 10:30:30,-20000,RUR,-268.244301


## Задача 2. Счет пользователя

Необходимо узнать итоговый баланс каждого пользовтеля.

### Исходные данные

Таблица `users`

In [ ]:
users = pd.DataFrame([
    [1, 'Alice', 'alice@example.com', '89151234567'],
    [2, 'Bob', 'bob@example.com', '+79167654321'],
    [3, 'Charlie', 'ch@example.com', '8(985) 123-45-67'],
    [4, 'Dylan', 'dylan@example.com', '+79167654321'],
    [5, 'Eve', 'eve@example.com', '+79167654321'],
    [6, 'Frank', 'frank@example.com', '+79851234567'],
    [7, 'Glenda', 'glenda@example.com', '+12124504567']
], columns=['acc', 'name', 'email', 'phone'])

users

,acc,name,email,phone
0,1,Alice,alice@example.com,89151234567
1,2,Bob,bob@example.com,+79167654321
2,3,Charlie,ch@example.com,8(985) 123-45-67
3,4,Dylan,dylan@example.com,+79167654321
4,5,Eve,eve@example.com,+79167654321
5,6,Frank,frank@example.com,+79851234567
6,7,Glenda,glenda@example.com,+12124504567


Таблица `transactions`
- `from_acc` - номер счета отправителя
- `to_acc` - номер счета получателя

если `from_acc` = -1, это был ввод денег в платежную систему, `to_acc` = -1 соответственно, вывод.


In [ ]:
transactions = pd.DataFrame([
    [1, -1, 1, 100, '2020-12-31'],
    [2, -1, 2, 200, '2020-12-31'],
    [3, -1, 3, 250, '2020-12-31'],
    [4, -1, 4, 1500, '2020-12-31'],
    [5, -1, 5, 20, '2020-12-31'],
    [6, 1, 2, 10.5, '2021-01-23'],
    [7, 2, 1, 11, '2021-01-24'],
    [8, 1, 3, 5, '2021-02-10'],
    [9, 2, 3, 5, '2021-02-11'],
    [10, 3, 7, 8, '2021-02-12'],
    [11, 4, 3, 20, '2021-03-05'],
    [12, 5, 3, 20, '2021-03-06'],
    [13, 1, 3, 20, '2021-03-29'],
    [14, 3, 7, 50, '2021-04-05'],
    [15, 7, -1, 58, '2021-04-12']
], columns=['id', 'from_acc', 'to_acc', 'amount', 'dt'])

transactions['dt'] = pd.to_datetime(transactions['dt'])
transactions

,id,from_acc,to_acc,amount,dt
0,1,-1,1,100.0,2020-12-31
1,2,-1,2,200.0,2020-12-31
2,3,-1,3,250.0,2020-12-31
3,4,-1,4,1500.0,2020-12-31
4,5,-1,5,20.0,2020-12-31
5,6,1,2,10.5,2021-01-23
6,7,2,1,11.0,2021-01-24
7,8,1,3,5.0,2021-02-10
8,9,2,3,5.0,2021-02-11
9,10,3,7,8.0,2021-02-12


In [ ]:
pd.merge(users[['acc', 'name']],
    transactions.groupby('to_acc')['amount'].sum() - 
    transactions.groupby('from_acc')['amount'].sum(),
    right_index=True, left_on='acc', how='left'
).fillna(0)

,acc,name,amount
0,1,Alice,75.5
1,2,Bob,194.5
2,3,Charlie,262.0
3,4,Dylan,1480.0
4,5,Eve,0.0
5,6,Frank,0.0
6,7,Glenda,0.0


In [ ]:
transactions.to_sql('transactions', con, index=False, if_exists='replace')
users.to_sql('users', con, index=False, if_exists='replace')

In [ ]:
query = '''
WITH incomes AS (
  SELECT to_acc AS acc, SUM(amount) amount 
  FROM transactions GROUP BY to_acc
),
consumptions AS (
  SELECT from_acc AS acc, SUM(amount) amount
  FROM transactions GROUP BY from_acc
)
SELECT
  u.acc
  , u.name
  , COALESCE(i.amount - c.amount, 0) AS balance
FROM users u
LEFT JOIN consumptions c
  ON u.acc == c.acc
LEFT JOIN incomes i
  ON u.acc == i.acc
'''

select(query)

,acc,name,balance
0,1,Alice,75.5
1,2,Bob,194.5
2,3,Charlie,262.0
3,4,Dylan,1480.0
4,5,Eve,0.0
5,6,Frank,0.0
6,7,Glenda,0.0


## Задача 3. Раскрыть скобки

Дана строка. Предстваить содержимое [] в виде последовательности, повторенной n раз, равному числу перед скобками.

Примеры:
```
a2[bc] = abcbc
a2[bc3[g]] = abcgggbcggg
2[ab3[gc4[d]]] = abgcddddgcddddgcddddabgcddddgcddddgcdddd
a2[b1[c]] = abcbc
```

In [27]:
import re

pattern = re.compile(r'(\d+)?\[([^[\]]*)\]')
# на случай отсутствия числа перед скобками, оно замещается единицей

def unbracket(s, pattern):
    while pattern.subn('', s)[1]:
        s = pattern.sub(lambda x: (int(x[1]) or 1) * x[2], s)
    return s

assert unbracket('a2[bc]', pattern) == 'abcbc'
assert unbracket('a2[bc3[g]]', pattern) == 'abcgggbcggg'
assert unbracket('2[ab3[gc4[d]]]', pattern) == 'abgcddddgcddddgcddddabgcddddgcddddgcdddd'
assert unbracket('a2[b1[c]]', pattern) == 'abcbc'
print('All\'s ok')

All's ok
